## Importando Libs

In [114]:
import pandas as pd
import asyncio
import aiohttp
import json
from tqdm import tqdm
import re


## Importando Dados

In [115]:
df_usuario = pd.read_csv("./mnt/data/usuarios.csv", delimiter=",")
df_incidente = pd.read_csv("./mnt/data/incidentes.csv", delimiter=",")
df_ativo = pd.read_csv("./mnt/data/ativos_monitorados.csv", delimiter=",")
df_plataforma = pd.read_csv("./mnt/data/plataformas.csv", delimiter=",")
df_antivirus = pd.read_csv("./mnt/data/antivirus.csv", delimiter=",")
import nest_asyncio
nest_asyncio.apply()


# Tratamento de dados

#### 1. Tabela Usuario

In [116]:
# Visualização dos 5 primeiros
df_usuario.head()

,id_usuario,nome,idade,cep,data_de_nascimento,renda
0,1,Sarah Caldeira,49,05404-010,1975-11-09,13431.34
1,2,Dr. Luiz Otávio Vargas,42,04105-010,1983-04-28,9792.60
2,3,Srta. Marcela Pereira,28,02862-120,1996-10-20,1929.75
3,4,Dr. Luigi Garcia,27,04836-110,1997-05-22,2267.47
4,5,Antonella da Mata,60,04836-110,1965-01-08,2906.67


##### Limpeza

In [117]:
# Verificando se há dados nulos
display(df_usuario.isnull().sum())

# Verificando se há duplicatas
display(df_usuario.duplicated().sum())

id_usuario            0
nome                  0
idade                 0
cep                   0
data_de_nascimento    0
renda                 0
dtype: int64

np.int64(0)

##### Obtém bairro, cidade e estado a partir do CEP

In [118]:
# # Validação de CEP (apenas números, 8 dígitos)
# def is_cep_valido(cep):
#     return re.fullmatch(r'\d{8}', cep) is not None

# # Função assíncrona para buscar CEP
# async def fetch_cep(session, cep):
#     url = f'https://viacep.com.br/ws/{cep}/json/'
#     try:
#         async with session.get(url) as resp:
#             if resp.status == 200:
#                 data = await resp.json()
#                 if 'erro' not in data:
#                     return cep, {
#                         'bairro': data.get('bairro', ''),
#                         'cidade': data.get('localidade', ''),
#                         'estado': data.get('uf', '')
#                     }
#     except:
#         pass
#     return cep, {'bairro': '', 'cidade': '', 'estado': ''}

# # Consulta em lote com barra de progresso
# async def consultar_ceps_em_lote(ceps):
#     cep_cache = {}
#     async with aiohttp.ClientSession() as session:
#         tasks = [fetch_cep(session, cep) for cep in ceps]
#         for future in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Consultando CEPs"):
#             cep, resultado = await future
#             cep_cache[cep] = resultado
#     return cep_cache

# # Limpeza dos CEPs no DataFrame
# df_usuario['cep'] = df_usuario['cep'].astype(str).str.replace('-', '', regex=False).str.strip()

# # Carrega cache salvo, se existir
# try:
#     with open('cep_cache.json', 'r', encoding='utf-8') as f:
#         cep_cache = json.load(f)
# except FileNotFoundError:
#     cep_cache = {}

# # Gera lista de CEPs únicos e válidos ainda não consultados
# ceps_unicos = df_usuario['cep'].dropna().unique()
# ceps_validos = [cep for cep in ceps_unicos if is_cep_valido(cep)]
# ceps_para_buscar = [cep for cep in ceps_validos if cep not in cep_cache]

# # Consulta os CEPs novos e atualiza o cache
# if ceps_para_buscar:
#     novos_resultados = await consultar_ceps_em_lote(ceps_para_buscar)
#     cep_cache.update(novos_resultados)
#     with open('cep_cache.json', 'w', encoding='utf-8') as f:
#         print(f)
#         json.dump(cep_cache, f, ensure_ascii=False)

# # Aplica os dados ao DataFrame
# df_localizacao = df_usuario['cep'].map(cep_cache).apply(pd.Series)
# df_usuario = df_usuario.drop(columns='cep')
# df_usuario = pd.concat([df_usuario, df_localizacao], axis=1)
